In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import os
from tensorflow.keras.callbacks import ModelCheckpoint

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("./Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], 1)
application_df

C:\Users\chefc\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [3]:
# Determine the number of unique values in each column.
#print(application_df.nunique())
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_df[application_cat].nunique()

APPLICATION_TYPE          17
AFFILIATION                6
CLASSIFICATION            71
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
dtype: int64

In [4]:
app_counts = application_df.APPLICATION_TYPE.value_counts()
app_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Determine which values to replace if counts are less than ...?
replace_application = list(app_counts[app_counts < 528].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
class_counts = application_df.CLASSIFICATION.value_counts()
class_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# Determine which values to replace if counts are less than ..?
replace_class = list(class_counts[class_counts < 1883].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [8]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_df[application_cat].nunique()

APPLICATION_TYPE          9
AFFILIATION               6
CLASSIFICATION            6
USE_CASE                  5
ORGANIZATION              4
INCOME_AMT                9
SPECIAL_CONSIDERATIONS    2
dtype: int64

In [9]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

C:\Users\chefc\anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True,right_index=True).drop(application_cat,1)

application_df.head()

C:\Users\chefc\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [11]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify = y)

C:\Users\chefc\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3520      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5,981
Trainable params: 5,981
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [21]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
804/804 [==============================] - 1s 656us/step - loss: 0.5348 - accuracy: 0.7407
Epoch 2/100
804/804 [==============================] - 1s 653us/step - loss: 0.5349 - accuracy: 0.7412
Epoch 3/100
804/804 [==============================] - 1s 657us/step - loss: 0.5348 - accuracy: 0.7414
Epoch 4/100
804/804 [==============================] - 1s 652us/step - loss: 0.5350 - accuracy: 0.7416
Epoch 5/100
804/804 [==============================] - 1s 646us/step - loss: 0.5343 - accuracy: 0.7408
Epoch 6/100
804/804 [==============================] - 1s 643us/step - loss: 0.5344 - accuracy: 0.7415
Epoch 7/100
804/804 [==============================] - 1s 645us/step - loss: 0.5343 - accuracy: 0.7411
Epoch 8/100
804/804 [==============================] - 1s 666us/step - loss: 0.5345 - accuracy: 0.7414
Epoch 9/100
804/804 [==============================] - 1s 663us/step - loss: 0.5342 - accuracy: 0.7414
Epoch 10/100
804/804 [==============================] - 1s 661us/step - l

804/804 [==============================] - 1s 748us/step - loss: 0.5324 - accuracy: 0.7416
Epoch 80/100
804/804 [==============================] - 1s 673us/step - loss: 0.5324 - accuracy: 0.7426
Epoch 81/100
804/804 [==============================] - 1s 684us/step - loss: 0.5324 - accuracy: 0.7422
Epoch 82/100
804/804 [==============================] - 1s 669us/step - loss: 0.5323 - accuracy: 0.7421
Epoch 83/100
804/804 [==============================] - 1s 684us/step - loss: 0.5322 - accuracy: 0.7410
Epoch 84/100
804/804 [==============================] - 1s 669us/step - loss: 0.5323 - accuracy: 0.7423
Epoch 85/100
804/804 [==============================] - 1s 652us/step - loss: 0.5323 - accuracy: 0.7420
Epoch 86/100
804/804 [==============================] - 1s 658us/step - loss: 0.5326 - accuracy: 0.7420
Epoch 87/100
804/804 [==============================] - 1s 658us/step - loss: 0.5323 - accuracy: 0.7416
Epoch 88/100
804/804 [==============================] - 1s 648us/step - loss:

### Optimazation Attempt 1, Adding Additional Hidden Layer

In [22]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 10

nn1 = tf.keras.models.Sequential()

# First hidden layer
nn1.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn1.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Second hidden layer
nn1.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn1.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 80)                3520      
                                                                 
 dense_8 (Dense)             (None, 30)                2430      
                                                                 
 dense_9 (Dense)             (None, 10)                310       
                                                                 
 dense_10 (Dense)            (None, 1)                 11        
                                                                 
Total params: 6,271
Trainable params: 6,271
Non-trainable params: 0
_________________________________________________________________


In [23]:
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [24]:
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5*804
    )

In [25]:
nn1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [26]:
# Train the model
fit_model = nn1.fit(X_train_scaled,y_train,epochs=100, callbacks=[cp_callback])
model_loss, model_accuracy = nn1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
804/804 [==============================] - 1s 699us/step - loss: 0.5706 - accuracy: 0.7204
Epoch 2/100
804/804 [==============================] - 1s 699us/step - loss: 0.5546 - accuracy: 0.7307
Epoch 3/100
804/804 [==============================] - 1s 678us/step - loss: 0.5523 - accuracy: 0.7327
Epoch 4/100
804/804 [==============================] - 1s 683us/step - loss: 0.5501 - accuracy: 0.7325
Epoch 5/100
767/804 [===========================>..] - ETA: 0s - loss: 0.5489 - accuracy: 0.7340
Epoch 5: saving model to checkpoints\weights.05.hdf5
804/804 [==============================] - 1s 822us/step - loss: 0.5492 - accuracy: 0.7338
Epoch 6/100
804/804 [==============================] - 1s 727us/step - loss: 0.5479 - accuracy: 0.7347
Epoch 7/100
804/804 [==============================] - 1s 738us/step - loss: 0.5470 - accuracy: 0.7360
Epoch 8/100
804/804 [==============================] - 1s 727us/step - loss: 0.5467 - accuracy: 0.7352
Epoch 9/100
804/804 [=================

804/804 [==============================] - 1s 688us/step - loss: 0.5368 - accuracy: 0.7400
Epoch 64/100
804/804 [==============================] - 1s 694us/step - loss: 0.5360 - accuracy: 0.7412
Epoch 65/100
765/804 [===========================>..] - ETA: 0s - loss: 0.5361 - accuracy: 0.7413
Epoch 65: saving model to checkpoints\weights.65.hdf5
804/804 [==============================] - 1s 685us/step - loss: 0.5358 - accuracy: 0.7405
Epoch 66/100
804/804 [==============================] - 1s 669us/step - loss: 0.5360 - accuracy: 0.7405
Epoch 67/100
804/804 [==============================] - 1s 669us/step - loss: 0.5358 - accuracy: 0.7409
Epoch 68/100
804/804 [==============================] - 1s 679us/step - loss: 0.5359 - accuracy: 0.7411
Epoch 69/100
804/804 [==============================] - 1s 672us/step - loss: 0.5358 - accuracy: 0.7414
Epoch 70/100
755/804 [===========================>..] - ETA: 0s - loss: 0.5367 - accuracy: 0.7412
Epoch 70: saving model to checkpoints\weights.70

### Optimzation Attempt 2, Adding Neurons

In [27]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 200
hidden_nodes_layer2 = 100
hidden_nodes_layer3 = 50

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn2.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 200)               8800      
                                                                 
 dense_12 (Dense)            (None, 100)               20100     
                                                                 
 dense_13 (Dense)            (None, 50)                5050      
                                                                 
 dense_14 (Dense)            (None, 1)                 51        
                                                                 
Total params: 34,001
Trainable params: 34,001
Non-trainable params: 0
_________________________________________________________________


In [28]:
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [29]:
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5*804
    )

In [30]:
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [31]:
# Train the model
fit_model = nn2.fit(X_train_scaled,y_train,epochs=100, callbacks=[cp_callback])
model_loss, model_accuracy = nn1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
804/804 [==============================] - 1s 817us/step - loss: 0.5679 - accuracy: 0.7230
Epoch 2/100
804/804 [==============================] - 1s 811us/step - loss: 0.5542 - accuracy: 0.7308
Epoch 3/100
804/804 [==============================] - 1s 781us/step - loss: 0.5515 - accuracy: 0.7332
Epoch 4/100
804/804 [==============================] - 1s 918us/step - loss: 0.5501 - accuracy: 0.7332
Epoch 5/100
745/804 [==========================>...] - ETA: 0s - loss: 0.5474 - accuracy: 0.7344
Epoch 5: saving model to checkpoints\weights.05.hdf5
804/804 [==============================] - 1s 835us/step - loss: 0.5482 - accuracy: 0.7336
Epoch 6/100
804/804 [==============================] - 1s 836us/step - loss: 0.5481 - accuracy: 0.7337
Epoch 7/100
804/804 [==============================] - 1s 789us/step - loss: 0.5469 - accuracy: 0.7354
Epoch 8/100
804/804 [==============================] - 1s 797us/step - loss: 0.5465 - accuracy: 0.7354
Epoch 9/100
804/804 [=================

804/804 [==============================] - 1s 745us/step - loss: 0.5335 - accuracy: 0.7419
Epoch 64/100
804/804 [==============================] - 1s 748us/step - loss: 0.5336 - accuracy: 0.7421
Epoch 65/100
753/804 [===========================>..] - ETA: 0s - loss: 0.5342 - accuracy: 0.7406
Epoch 65: saving model to checkpoints\weights.65.hdf5
804/804 [==============================] - 1s 766us/step - loss: 0.5336 - accuracy: 0.7417
Epoch 66/100
804/804 [==============================] - 1s 745us/step - loss: 0.5334 - accuracy: 0.7417
Epoch 67/100
804/804 [==============================] - 1s 773us/step - loss: 0.5329 - accuracy: 0.7420
Epoch 68/100
804/804 [==============================] - 1s 744us/step - loss: 0.5338 - accuracy: 0.7415
Epoch 69/100
804/804 [==============================] - 1s 745us/step - loss: 0.5334 - accuracy: 0.7416
Epoch 70/100
752/804 [===========================>..] - ETA: 0s - loss: 0.5325 - accuracy: 0.7426
Epoch 70: saving model to checkpoints\weights.70

### Optimzation Attempt 3, increasing EPOCHs

In [32]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 200
hidden_nodes_layer2 = 100
hidden_nodes_layer3 = 50

nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn3.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 200)               8800      
                                                                 
 dense_16 (Dense)            (None, 100)               20100     
                                                                 
 dense_17 (Dense)            (None, 50)                5050      
                                                                 
 dense_18 (Dense)            (None, 1)                 51        
                                                                 
Total params: 34,001
Trainable params: 34,001
Non-trainable params: 0
_________________________________________________________________


In [33]:
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [34]:
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5*804
    )

In [35]:
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [36]:
# Train the model
fit_model = nn3.fit(X_train_scaled,y_train,epochs=200, callbacks=[cp_callback])
model_loss, model_accuracy = nn1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/200
804/804 [==============================] - 1s 797us/step - loss: 0.5669 - accuracy: 0.7255
Epoch 2/200
804/804 [==============================] - 1s 790us/step - loss: 0.5545 - accuracy: 0.7313
Epoch 3/200
804/804 [==============================] - 1s 791us/step - loss: 0.5514 - accuracy: 0.7315
Epoch 4/200
804/804 [==============================] - 1s 790us/step - loss: 0.5504 - accuracy: 0.7345
Epoch 5/200
780/804 [============================>.] - ETA: 0s - loss: 0.5483 - accuracy: 0.7339
Epoch 5: saving model to checkpoints\weights.05.hdf5
804/804 [==============================] - 1s 806us/step - loss: 0.5482 - accuracy: 0.7335
Epoch 6/200
804/804 [==============================] - 1s 787us/step - loss: 0.5470 - accuracy: 0.7352
Epoch 7/200
804/804 [==============================] - 1s 798us/step - loss: 0.5468 - accuracy: 0.7359
Epoch 8/200
804/804 [==============================] - 1s 792us/step - loss: 0.5464 - accuracy: 0.7350
Epoch 9/200
804/804 [=================

804/804 [==============================] - 1s 790us/step - loss: 0.5338 - accuracy: 0.7412
Epoch 64/200
804/804 [==============================] - 1s 776us/step - loss: 0.5338 - accuracy: 0.7413
Epoch 65/200
791/804 [============================>.] - ETA: 0s - loss: 0.5334 - accuracy: 0.7413
Epoch 65: saving model to checkpoints\weights.65.hdf5
804/804 [==============================] - 1s 796us/step - loss: 0.5341 - accuracy: 0.7407
Epoch 66/200
804/804 [==============================] - 1s 777us/step - loss: 0.5338 - accuracy: 0.7414
Epoch 67/200
804/804 [==============================] - 1s 781us/step - loss: 0.5331 - accuracy: 0.7415
Epoch 68/200
804/804 [==============================] - 1s 777us/step - loss: 0.5338 - accuracy: 0.7419
Epoch 69/200
804/804 [==============================] - 1s 779us/step - loss: 0.5340 - accuracy: 0.7418
Epoch 70/200
788/804 [============================>.] - ETA: 0s - loss: 0.5335 - accuracy: 0.7416
Epoch 70: saving model to checkpoints\weights.70

781/804 [============================>.] - ETA: 0s - loss: 0.5316 - accuracy: 0.7423
Epoch 125: saving model to checkpoints\weights.125.hdf5
804/804 [==============================] - 1s 801us/step - loss: 0.5314 - accuracy: 0.7428
Epoch 126/200
804/804 [==============================] - 1s 778us/step - loss: 0.5313 - accuracy: 0.7425
Epoch 127/200
804/804 [==============================] - 1s 786us/step - loss: 0.5313 - accuracy: 0.7427
Epoch 128/200
804/804 [==============================] - 1s 815us/step - loss: 0.5312 - accuracy: 0.7425
Epoch 129/200
804/804 [==============================] - 1s 814us/step - loss: 0.5317 - accuracy: 0.7419
Epoch 130/200
782/804 [============================>.] - ETA: 0s - loss: 0.5310 - accuracy: 0.7431
Epoch 130: saving model to checkpoints\weights.130.hdf5
804/804 [==============================] - 1s 857us/step - loss: 0.5309 - accuracy: 0.7430
Epoch 131/200
804/804 [==============================] - 1s 805us/step - loss: 0.5338 - accuracy: 0.74

Epoch 186/200
804/804 [==============================] - 1s 784us/step - loss: 0.5307 - accuracy: 0.7432
Epoch 187/200
804/804 [==============================] - 1s 795us/step - loss: 0.5301 - accuracy: 0.7428
Epoch 188/200
804/804 [==============================] - 1s 782us/step - loss: 0.5301 - accuracy: 0.7429
Epoch 189/200
804/804 [==============================] - 1s 845us/step - loss: 0.5298 - accuracy: 0.7432
Epoch 190/200
756/804 [===========================>..] - ETA: 0s - loss: 0.5293 - accuracy: 0.7431
Epoch 190: saving model to checkpoints\weights.190.hdf5
804/804 [==============================] - 1s 816us/step - loss: 0.5299 - accuracy: 0.7432
Epoch 191/200
804/804 [==============================] - 1s 795us/step - loss: 0.5304 - accuracy: 0.7429
Epoch 192/200
804/804 [==============================] - 1s 804us/step - loss: 0.5305 - accuracy: 0.7430
Epoch 193/200
804/804 [==============================] - 1s 779us/step - loss: 0.5299 - accuracy: 0.7437
Epoch 194/200
804/8

In [37]:
# Export our model to HDF5 file
nn3.save("AlphabetSoupCharity_Optimzation.h5")